In [219]:
from octopus import OctopusClient

import networkx as nx

In [220]:
def build_nx_graph():
    """
    Builds and returns networkx graph for Movies.

    Parameters : 
    None

    Returns : 
    networkx graph object
        A networkx graph object of Movies
    """
    import csv 
    
    def remove_escape_quotes(s):
        return s.replace('\\"', '"')

    movies_csv_path = "../utils/movies.csv"
    G = nx.Graph()

    with open(movies_csv_path, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)
        for row in reader:
            # Check if it's a node or relationship
            if row[0]:  # Node
                #print(row[0])
                node_type = row[1].split(":")[1]  # Extract node type from labels
                if node_type == "Person":
                    G.add_node(row[0], label="Person", born=row[2], name=remove_escape_quotes(row[3]))
                elif node_type == "Movie":
                    G.add_node(row[0], label="Movie", released=row[4], tagline=remove_escape_quotes(row[5]), title=remove_escape_quotes(row[6]))
            else:  # Relationship
                rel_type = row[9]
                if rel_type == "ACTED_IN":
                    G.add_edge(row[7], row[8], label="ACTED_IN", rating=remove_escape_quotes(row[10]), roles=remove_escape_quotes(row[11]))
                elif rel_type == "WROTE":
                    G.add_edge(row[7], row[8], label="WROTE")
                elif rel_type == "PRODUCED":
                    G.add_edge(row[7], row[8], label="PRODUCED")
                elif rel_type == "DIRECTED":
                    G.add_edge(row[7], row[8], label="DIRECTED")
                elif rel_type == "REVIEWED":
                    G.add_edge(row[7], row[8], label="REVIEWED", rating=remove_escape_quotes(row[10]), summary=remove_escape_quotes(row[11]))

    return G
    

In [221]:
def task_1_nx(nx_graph):
    """
    Using networkx library, find all patterns between “Tom Hanks”, “Helen Hunt”, and “Kevin Bacon”.

    Parameters:
    nx_graph : networkx object
       Movies graph
    
    Returns :
    None
    """
    def get_node(name):
        for node, attrs in nx_graph.nodes(data=True):
            if attrs.get("name") == name:
                return node
    
    th = get_node("Tom Hanks")
    hh = get_node("Helen Hunt")
    kb = get_node("Kevin Bacon")
    
    patterns = []
    for actor1 in [th, hh, kb]:
        for actor2 in [th, hh, kb]:
            if actor1 != actor2:
                for path in nx.all_shortest_paths(nx_graph, source=actor1, target=actor2):
                    patterns.append(path)

    # Print the found patterns
    for pattern in patterns:
        print(pattern)

In [222]:
def task_1_octopus(oc):
    """
    Using Octopus, find all patterns between “Tom Hanks”, “Helen Hunt”, and “Kevin Bacon”.

    Parameters:
    oc : Octopus Client
    
    Returns :
    None
    """
    query = "MATCH p = (actor1:Person)-[*]-(actor2:Person) WHERE actor1.name IN [\"Tom Hanks\", \"Helen Hunt\", \"Kevin Bacon\"] \
            AND actor2.name IN [\"Tom Hanks\", \"Helen Hunt\", \"Kevin Bacon\"] \
            AND actor1 <> actor2 \
            RETURN nodes(p) AS pattern"
    execution = oc.execute(query, 1)
    execution.poll()
    print(execution.output.fetch_all())
    

In [223]:
def task_2_nx(nx_graph):
    """
    Using networkx library, find movies and actors up to 4 “hops” away from “Kevin Bacon” (including "Kevin Bacon").


    Parameters:
    nx_graph : networkx object
       Movies graph
    
    Returns :
    None
    """
    # Find all nodes 4 hops away from "Kevin Bacon"
    kb = None
    for node, attrs in nx_graph.nodes(data=True):
       if attrs.get("name") == "Kevin Bacon":
          kb = node
          break
    
    # Find movies and persons up to 4 hops away from "Kevin Bacon"
    nodes_4_hops_away = set()
    for neighbor1 in nx_graph.neighbors(kb):
        nodes_4_hops_away.add(neighbor1)
        for neighbor2 in nx_graph.neighbors(neighbor1):
            nodes_4_hops_away.add(neighbor2)
            for neighbor3 in nx_graph.neighbors(neighbor2):
                nodes_4_hops_away.add(neighbor3)
                for neighbor4 in nx_graph.neighbors(neighbor3):
                    if neighbor4 != kb:
                       nodes_4_hops_away.add(neighbor4)

    # Extract movies and persons from the set
    movies_and_persons = list(nodes_4_hops_away)

    # Print
    for x in movies_and_persons:
        for node, attrs in nx_graph.nodes(data=True):
           if node == x:
              print(attrs)
    
    print(len(movies_and_persons))

In [224]:
def task_2_octopus(oc):
    """
    Using Octopus, find movies and actors up to 4 “hops” away from “Kevin Bacon”.

    Parameters:
    oc : Octopus Client
    
    Returns :
    None
    """
    query = "MATCH (bacon:Person {name:\"Kevin Bacon\"})-[*1..4]-(hollywood) \
            RETURN DISTINCT hollywood"    
    execution = oc.execute(query, 1)
    execution.poll()
    print(execution.output.fetch_all())

In [225]:
def task_3_nx(nx_graph):
    """
    Using networkx library, find the shortest path from “Kevin Bacon” to “Meg Ryan”.

    Parameters:
    nx_graph : networkx object
       Movies graph
    
    Returns :
    None
    """
    def get_node(name):
        for node, attrs in nx_graph.nodes(data=True):
            if attrs.get("name") == name:
                return node

    kb = get_node("Kevin Bacon")
    mr = get_node("Meg Ryan")
    
    shortest_path = nx.shortest_path(nx_graph, source=kb, target=mr)
    print(shortest_path)

In [226]:
def task_3_octopus(oc):
    """
    Using Octopus, find the shortest path from “Kevin Bacon” to “Meg Ryan”.

    Parameters:
    oc : Octopus Client
    
    Returns :
    None
    """
    query = "MATCH p=shortestPath( \
            (bacon:Person {name:\"Kevin Bacon\"})-[*]-(meg:Person {name:\"Meg Ryan\"})) \
            RETURN p"   
    execution = oc.execute(query, 1)
    execution.poll()
    print(execution.output.fetch_all())

In [227]:
if __name__ == "__main__":
    # Build Networkx Graph
    nx_graph = build_nx_graph()

    # Instantiate Octopus Client Object
    oc = OctopusClient("localhost", "8000", "neo4j", "password")

    # Task-1
    #print("Task-1: Find all patterns between \"Tom Hanks\", \"Helen Hunt\", and \"Kevin Bacon\"")
    #print("networkx:")
    #task_1_nx(nx_graph)
    #print("Octopus:")
    #task_1_octopus(oc)

    # Task-2
    #print("Task-2: Find movies and actors up to 4 \"hops\" away from \"Kevin Bacon\" (including \"Kevin Bacon\")")
    #print("networkx:")
    #task_2_nx(nx_graph)
    #print("Octopus:")
    #task_2_octopus(oc)

    # Task-3
    #print("Task-3: Find the shortest path from \"Kevin Bacon\" to \"Meg Ryan\"")
    #print("networkx:")
    #task_3_nx(nx_graph)
    #print("Octopus:")
    #task_3_octopus(oc)